In [1]:
import kfp
import yaml
import os
import logging
import fairing

from kubernetes import client as k8s_client
from kubernetes import config as k8s_config
from pathlib import Path
GCP_PROJECT = fairing.cloud.gcp.guess_project_name()
try: 
    namespace = fairing.utils.get_current_k8s_namespace()
except FileNotFoundError as e: 
    namespace="kubeflow"
logging.info(f"Running in project {GCP_PROJECT}")
logging.info(f"Running in namespace {namespace}")

/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
Running in project ds-playground-237314
Running in namespace kubeflow


In [2]:
#! pip install kfp
#! pip install table_logger
#! pip install fairing
#! pip install retrying

# Notebook to start mpi-operator job

In [3]:
!export GOOGLE_APPLICATION_CREDENTIALS=""

## Define mpi job. 

This example is based aroud the tensorflow-benchmark found [here](https://github.com/kubeflow/mpi-operator/blob/master/examples/v1alpha1/tensorflow-benchmarks.yaml). To make this example we have defined the complete job below.

In [4]:
mpi_job = f"""
apiVersion: kubeflow.org/v1alpha2
kind: MPIJob
metadata:
  name: mpi-test
spec:
  slotsPerWorker: 1
  cleanPodPolicy: Running
  mpiReplicaSpecs:
    Launcher:
      replicas: 1
      template:
        spec:
          containers:
          - image: nikenano/horovod:latest
            name: mpi-launcher
            command:
            - mpirun
            args:
            - -np
            - "2"
            - --allow-run-as-root
            - -bind-to
            - none
            - -map-by
            - slot
            - -x
            - LD_LIBRARY_PATH
            - -x
            - PATH
            - -mca
            - pml
            - ob1
            - -mca
            - btl
            - ^openib
            - python
            - /examples/tensorflow_mnist.py
            resources:
              limits:
                cpu: 1
                memory: 2Gi
    Worker:
      replicas: 2
      template:
        spec:
          containers:
          - image: nikenano/horovod:latest
            name: mpi-worker
            resources:
              limits:
                cpu: 2
                memory: 4Gi
"""

## Helper function to watch Kubernetes

In [5]:
import retrying
from kubernetes import client
from kubernetes import watch as k8s_watch
from table_logger import TableLogger


tbl = TableLogger(
  columns='NAME,STATE,TIME',
  colwidth={'NAME': 30, 'STATE':20, 'TIME':30},
  border=False)

@retrying.retry(wait_fixed=1000, stop_max_attempt_number=20)
def watch_k8s(name=None, namespace=None, timeout_seconds=600):
  """Watch the created or patched InferenceService in the specified namespace"""

  #if namespace is None:
    #namespace = utils.get_default_target_namespace()

  stream = k8s_watch.Watch().stream(
    client.CustomObjectsApi().list_namespaced_custom_object,
    MPI_JOB_GROUP,
    MPI_JOB_VERSION,
    namespace,
    MPI_JOB_PLURAL,
    timeout_seconds=timeout_seconds)

  for event in stream:
    mpijob = event['object']
    mpijob_name = mpijob['metadata']['name']
    if name and name != mpijob_name:
      continue
    else:
      status = ''
      update_time = ''
      last_condition = mpijob.get('status', {}).get('conditions', [])[-1]
      status = last_condition.get('type', '')
      update_time = last_condition.get('lastTransitionTime', '')

      tbl(mpijob_name, status, update_time)

      if name == mpijob_name:
        if status == 'Succeeded' or status == 'Failed':
          break

### Setup a python mpijob client

This sectin contains some helper code that is a modified version of the [tfjob client](https://github.com/kubeflow/tf-operator/blob/master/sdk/python/docs/TFJobClient.md). 

In [6]:
from kubernetes import client, config
import time

def is_running_in_k8s():
  return os.path.isdir('/var/run/secrets/kubernetes.io/')

MPI_JOB_GROUP = "kubeflow.org"
MPI_JOB_PLURAL = "mpijobs"
MPI_JOB_NAME_LABEL = "job-mpi"
MPI_JOB_VERSION = "v1alpha2"
APISERVER_TIMEOUT = 120

class MPIJobClient(object):

  def __init__(self, config_file=None, context=None, # pylint: disable=too-many-arguments
               client_configuration=None, persist_config=True):
    """
    MPIJob client constructor
    :param config_file: kubeconfig file, defaults to ~/.kube/config
    :param context: kubernetes context
    :param client_configuration: kubernetes configuration object
    :param persist_config:
    """
    if config_file or not is_running_in_k8s():
      config.load_kube_config()
        #config_file=config_file,
        #context=context,
        #client_configuration=client_configuration,
        #persist_config=persist_config)
    else:
      config.load_incluster_config()

    self.custom_api = client.CustomObjectsApi()
    self.core_api = client.CoreV1Api()

  def create(self, mpijob, namespace=None):
    """
    Create the MPIJob
    :param mpijob: mpijob object
    :param namespace: defaults to current or default namespace
    :return: created mpijob
    """
    
    try:
      outputs = self.custom_api.create_namespaced_custom_object(
        MPI_JOB_GROUP,
        MPI_JOB_VERSION,
        namespace,
        MPI_JOB_PLURAL,
        mpijob)
    except client.rest.ApiException as e:
      raise RuntimeError(
        "Exception when calling CustomObjectsApi->create_namespaced_custom_object:\
         %s\n" % e)

    return outputs


  def is_job_succeeded(self, name, namespace=None):
    """Returns true if the MPIJob succeeded; false otherwise.
    :param name: The MPIJob name.
    :param namespace: defaults to current or default namespace.
    :return: True or False
    """
    mpijob_status = self.get_job_status(name, namespace=namespace)
    return mpijob_status.lower() == "succeeded"


  def get_job_status(self, name, namespace=None):
    """Returns MPIJob status, such as Running, Failed or Succeeded.
    :param name: The MPIJob name.
    :param namespace: defaults to current or default namespace.
    :return: Object MPIJob status
    """
    if namespace is None:
      namespace = utils.get_default_target_namespace()

    mpijob = self.get(name, namespace=namespace)
    last_condition = mpijob.get("status", {}).get("conditions", [])[-1]
    return last_condition.get("type", "")

  def wait_for_condition(self, name,
                         expected_condition,
                         namespace=None,
                         timeout_seconds=1200,
                         polling_interval=30,
                         status_callback=None):
    """Waits until any of the specified conditions occur.
    :param name: Name of the job.
    :param expected_condition: A list of conditions. Function waits until any of the
           supplied conditions is reached.
    :param namespace: defaults to current or default namespace.
    :param timeout_seconds: How long to wait for the job.
    :param polling_interval: How often to poll for the status of the job.
    :param status_callback: (Optional): Callable. If supplied this callable is
           invoked after we poll the job. Callable takes a single argument which
           is the job.
    :return: Object MPIJob status
    """

    #if namespace is None:
    #  namespace = utils.get_default_target_namespace()

    for _ in range(round(timeout_seconds/polling_interval)):

      mpi_job = None
      mpi_job = self.get(name, namespace=namespace)

      if mpi_job:
        if status_callback:
          status_callback(mpi_job)

        # If we poll the CRD quick enough status won't have been set yet.
        conditions = mpi_job.get("status", {}).get("conditions", [])
        # Conditions might have a value of None in status.
        conditions = conditions or []
        for c in conditions:
          if c.get("type", "") in expected_condition:
            return mpi_job

      time.sleep(polling_interval)

    raise RuntimeError(
      "Timeout waiting for MPIJob {0} in namespace {1} to enter one of the "
      "conditions {2}.".format(name, namespace, expected_condition), mpi_job)

  def get(self, name=None, namespace=None, watch=False, timeout_seconds=600): #pylint: disable=inconsistent-return-statements
    """
    Get the mpijob
    :param name: existing mpijob name, if not defined, the get all mpijobs in the namespace.
    :param namespace: defaults to current or default namespace
    :param watch: Watch the MPIJob if `True`.
    :param timeout_seconds: How long to watch the job..
    :return: mpijob
    """
    if namespace is None:
      namespace = utils.get_default_target_namespace()

    if name:
        thread = self.custom_api.get_namespaced_custom_object(
          MPI_JOB_GROUP,
          MPI_JOB_VERSION,
          namespace,
          MPI_JOB_PLURAL,
          name,
          async_req=True)

        mpijob = None
        try:
          mpijob = thread.get(APISERVER_TIMEOUT)
        except multiprocessing.TimeoutError:
          raise RuntimeError("Timeout trying to get MPIJob.")
        except client.rest.ApiException as e:
          raise RuntimeError(
            "Exception when calling CustomObjectsApi->get_namespaced_custom_object:\
            %s\n" % e)
        except Exception as e:
          raise RuntimeError(
            "There was a problem to get MPIJob {0} in namespace {1}. Exception: \
            {2} ".format(name, namespace, e))
        return mpijob
    else:
      if watch:
        watch_k8s(
            namespace=namespace,
            timeout_seconds=timeout_seconds)
      else:
        thread = self.custom_api.list_namespaced_custom_object(
          MPI_JOB_GROUP,
          MPI_JOB_VERSION,
          namespace,
          MPI_JOB_PLURAL,
          async_req=True)

        mpijob = None
        try:
          mpijobs = thread.get(constants.APISERVER_TIMEOUT)
        except multiprocessing.TimeoutError:
          raise RuntimeError("Timeout trying to get MPIJob.")
        except client.rest.ApiException as e:
          raise RuntimeError(
            "Exception when calling CustomObjectsApi->list_namespaced_custom_object:\
            %s\n" % e)
        except Exception as e:
          raise RuntimeError(
            "There was a problem to list MPIJobs in namespace {0}. \
            Exception: {1} ".format(namespace, e))
        return mpijobs

## MPI client

### Define the mpi client

In [7]:
#mpi_job_client = MPIJobClient(config_file="%s/.kube/config" % (str(Path.home()),)) # if you run localy
mpi_job_client = MPIJobClient()

### Run the job

In [8]:
mpi_job_body = yaml.safe_load(mpi_job)
mpi_job = mpi_job_client.create(mpi_job_body, namespace=namespace)  

logging.info(f"Created job in namespace: {namespace}")

Created job in namespace: kubeflow


### Wait for the job to finish

The job takes roughtly 20 minutes to finish if GPU resources needs to be provisioned. However if everything needed is allready provisined in the cluster it will be a lot faster.  

In [9]:
train_name="mpi-test"
mpi_job = mpi_job_client.wait_for_condition(train_name, expected_condition=["Succeeded", "Failed"], namespace=namespace, timeout_seconds=1200)

if mpi_job_client.is_job_succeeded(train_name, namespace):
    print("The job succeded")
    logging.info(f"MPIJob {namespace}.{train_name} succeeded")
else:
    raise ValueError(f"MPIJob {namespace}.{train_name} failed")

MPIJob kubeflow.mpi-test succeeded


The job succeded


### Check the logs from the job

In [10]:
os.environ["NAMESPACE"] = namespace
! kubectl -n ${NAMESPACE} logs -f $(kubectl get pods -n ${NAMESPACE} -l mpi_job_name=mpi-test,mpi_role_type=launcher -o name) -n ${NAMESPACE}

[mpi-test-launcher-8pqxp:00001] Warning: could not find environment variable "LD_LIBRARY_PATH"
+ POD_NAME=mpi-test-worker-0
+ shift
+ /opt/kube/kubectl exec mpi-test-worker-0 -- /bin/sh -c     PATH=/usr/local/bin:$PATH ; export PATH ; LD_LIBRARY_PATH=/usr/local/lib:$LD_LIBRARY_PATH ; export LD_LIBRARY_PATH ; DYLD_LIBRARY_PATH=/usr/local/lib:$DYLD_LIBRARY_PATH ; export DYLD_LIBRARY_PATH ;   /usr/local/bin/orted -mca ess "env" -mca ess_base_jobid "1846607872" -mca ess_base_vpid 1 -mca ess_base_num_procs "3" -mca orte_node_regex "mpi-test-launcher-[1:8]pqxp,mpi-test-worker-[1:0-1]@0(3)" -mca orte_hnp_uri "1846607872.0;tcp://10.48.0.77:39919" -mca pml "ob1" -mca btl "^openib" -mca plm "rsh" --tree-spawn -mca orte_parent_uri "1846607872.0;tcp://10.48.0.77:39919" -mca plm_rsh_agent "/etc/mpi/kubexec.sh" -mca orte_default_hostfile "/etc/mpi/hostfile" -mca hwloc_base_binding_policy "none" -mca rmaps_base_mapping_policy "slot" -mca pmix "^s1,s2,cray,isolated"
+ POD_NAME=mpi-test-worker-1
+ shif

## That was all! 